In [128]:
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [129]:
df = pd.read_csv("Bank_Personal_Loan_Modelling.csv")
df.drop(columns=['ID','ZIP Code'], inplace=True)
y = df.pop('Personal Loan')
x = df
x = StandardScaler().fit_transform(x)

In [130]:
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.25)

In [131]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
model = Sequential()
model.add(tf.keras.Input(shape=(11,)))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))

In [132]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
# Define the PSO parameters
n_particles = 10
n_iterations = 50
c1 = 0.5
c2 = 0.5
w = 0.9
v_max = 0.5


In [ ]:
# Define the fitness function
def fitness_function(weights):
    # Split the flattened weights into weight matrices
    shapes = [w.shape for w in model.get_weights()]
    weights = np.split(weights, np.cumsum([np.prod(s) for s in shapes])[:-1])
    weights = [w.reshape(s) for w, s in zip(weights, shapes)]

    # Set the weights of the model
    model.set_weights(weights)

    # Evaluate the model
    loss, acc = model.evaluate(x=x_ts, y=y_ts, verbose=0)
    return loss

In [ ]:
# Initialize the particles
positions = np.random.uniform(-1, 1, size=(n_particles, 417))
velocities = np.zeros_like(positions)

# Initialize the global best position and fitness
global_best_position = None
global_best_fitness = np.inf

In [ ]:
# Run the PSO algorithm
for i in range(n_iterations):
    # Evaluate the fitness of each particle
    fitness_values = np.array([fitness_function(p) for p in positions])
    
    # Update the global best position and fitness
    global_best_index = np.argmin(fitness_values)
    if fitness_values[global_best_index] < global_best_fitness:
        global_best_fitness = fitness_values[global_best_index]
        global_best_position = positions[global_best_index]
        
    # Update the velocities and positions of each particle
    for j in range(n_particles):
        r1 = np.random.uniform(0, 1)
        r2 = np.random.uniform(0, 1)
        
        local_best_position = positions[j]
        local_best_fitness = fitness_values[j]
        
        velocity = w * velocities[j] + c1 * r1 * (local_best_position - positions[j]) + c2 * r2 * (global_best_position - positions[j])
        velocity = np.clip(velocity, -v_max, v_max)
        
        positions[j] += velocity
        positions[j] = np.clip(positions[j], -1, 1)

In [133]:
model.fit(x=x_tr, y=y_tr, verbose=1, epochs=50)

Epoch 1/50
118/118 [==============================] - 0s 983us/step - loss: 0.4183 - accuracy: 0.8693
Epoch 2/50
118/118 [==============================] - 0s 874us/step - loss: 0.2097 - accuracy: 0.9317
Epoch 3/50
118/118 [==============================] - 0s 884us/step - loss: 0.1512 - accuracy: 0.9477
Epoch 4/50
118/118 [==============================] - 0s 904us/step - loss: 0.1267 - accuracy: 0.9576
Epoch 5/50
118/118 [==============================] - 0s 889us/step - loss: 0.1123 - accuracy: 0.9616
Epoch 6/50
118/118 [==============================] - 0s 967us/step - loss: 0.1024 - accuracy: 0.9664
Epoch 7/50
118/118 [==============================] - 0s 1ms/step - loss: 0.0952 - accuracy: 0.9683
Epoch 8/50
118/118 [==============================] - 0s 1ms/step - loss: 0.0900 - accuracy: 0.9691
Epoch 9/50
118/118 [==============================] - 0s 893us/step - loss: 0.0854 - accuracy: 0.9731
Epoch 10/50
118/118 [==============================] - 0s 846us/step - loss: 0.0817 - 

In [134]:
# Accuracy without using PSO
accuracy = model.evaluate(x=x_ts, y=y_ts, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")

Test accuracy: 97.6800%


In [ ]:
# Accuracy using PSO
accuracy = model.evaluate(x=x_ts, y=y_ts, verbose=0)[1]
print(f"Test accuracy: {accuracy*100:.4f}%")